# Latent Dirichlet Allocation

In [1]:
import pandas as pd

data = pd.read_csv('data', sep=",", header=None)

data.columns = ['text']

data.head()

,text
0,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...
1,From: atterlep@vela.acs.oakland.edu (Cardinal ...
2,From: miner@kuhub.cc.ukans.edu\nSubject: Re: A...
3,From: atterlep@vela.acs.oakland.edu (Cardinal ...
4,From: vzhivov@superior.carleton.ca (Vladimir Z...


The data is a collection of emails that are not labelled. Let's try extract topics from them!

## Preprocessing 

👇 You're used to it by now... Clean up! Store the cleaned text in a new dataframe column "clean_text".

In [2]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

def clean_text(df_column):
    clean_text = df_column.str.replace('[^\w\s]','').str.lower().str.replace('\d+', '') # ponctuation & lowercase & chiffres
    
    # supprime les stopwords 
    stop_words = stopwords.words('english')
    clean_text = clean_text.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    
    # lemmatize le texte
    lmtzr = WordNetLemmatizer()
    clean_text = clean_text.apply(lambda lst:[lmtzr.lemmatize(word) for word in lst])
    clean_text = clean_text.apply(lambda x : "".join(x))
    
    return clean_text

clean_text = clean_text(data['text'])
clean_text

C:\Users\Administrateur\AppData\Local\Temp\ipykernel_12576\946252299.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_text = df_column.str.replace('[^\w\s]','').str.lower().str.replace('\d+', '') # ponctuation & lowercase & chiffres


0       gldcunixbcccolumbiaedu gary l dare subject sta...
1       atterlepvelaacsoaklandedu cardinal ximenez sub...
2       minerkuhubccukansedu subject ancient books org...
3       atterlepvelaacsoaklandedu cardinal ximenez sub...
4       vzhivovsuperiorcarletonca vladimir zhivov subj...
                              ...                        
1194    jerrybeskimocom jerry kaufman subject prayers ...
1195    golchowyalchemychemutorontoca gerald olchowy s...
1196    jaynemmaltguildorg jayne kulikauskas subject q...
1197    sclarkepasutorontoca susan clark subject picks...
1198    lmvecwestminsteracuk william hargreaves subjec...
Name: text, Length: 1199, dtype: object

## Latent Dirichlet Allocation model

👇 Train an LDA model to extract potential topics.

In [3]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel

# Création d'un dictionnaire de mots et leur fréquence
dictionary = Dictionary(clean_text.str.split().tolist())

# Création d'une représentation de sac de mots du documents
bow_corpus = [dictionary.doc2bow(text) for text in clean_text.str.split().tolist()]

# Entrainement du modele LDA (Latent Dirichlet Allocation)
lda_model = LdaModel(bow_corpus, num_topics=10, id2word=dictionary, passes=50)

dictionary

Dans cet exemple, l'entrée du modèle LDA est une représentation de sac de mots des documents, qui est créée à l'aide de la bibliothèque **gensim**. L'argument **num_topics** spécifie le nombre de sujets que le **modèle LDA devrait extraire**. L'argument **id2word** fournit une correspondance entre les mots et leurs identificateurs, et l'argument passes spécifie le nombre de passages que le modèle devrait faire sur le corpus pendant l'entraînement.

## Visualize potential topics

👇 The function to print the words associated with the potential topics is already made for you. You just have to pass the correct arguments!

In [4]:
#!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# Préparation les données de visualisation
vis = gensimvis.prepare(lda_model, bow_corpus, dictionary)

# Affiche la visualisation
pyLDAvis.display(vis)

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


Dans cet exemple, la fonction **prepare** du module **pyLDAvis.gensim** est utilisée pour préparer les données de visualisation à partir du modèle LDA, du corpus de sac de mots et du dictionnaire. La fonction **display** du module **pyLDAvis** est ensuite utilisée pour afficher la visualisation.

## Predict topic of new text

👇 You can now use your LDA model to predict the topic of a new text. First, use your vectorizer to vectorize the example. Then, use your LDA model to predict the topic of the vectorized example.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
import gensim

# Instance de votre vectoriseur
vectorizer = CountVectorizer()

# nouveau texte
new_text = "this a exemple of text yeah come on"

def topic_distribution(string_input):
    string_input = [string_input]
    # Fit and transform
    X = vectorizer.fit_transform(string_input)
 
    # Convertir une matrice en corpus de gensim.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
 
    output = list(lda_model[corpus])[0]
 
    return output  
 
topic_distribution(new_text)


def topic_prediction(my_document):
    string_input = [my_document]
    X = vectorizer.transform(string_input)
    # Convertir une matrice en corpus de gensim
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
    output = list(lda_model[corpus])[0]
    topics = sorted(output,key=lambda x:x[1],reverse=True)
    return topics[0][0]
 
print("Sujet associé au nouveau texte: ", topic_prediction(new_text))


Sujet associé au nouveau texte:  7
